In [21]:
import requests
import json
import xmltodict
import urllib.parse
import opencitingpy
import datetime
import sqlite3
import os
from matplotlib import pyplot as plt
import numpy as np
import re
import math

# Setup sqlite Datenbank
Datenbank wird im Verzeichnis "C:\MA_Pethke_3992454" abgelegt

In [ ]:
# Anlegen von Datenbankspeicherort, falls er nicht existiert
path = "C:\MA_Pethke_3992454"
if not os.path.exists(path):
    os.makedirs(path)

# Verbindung zu DB aufbauen, DB wird automatisch erstellt, wenn keine da ist
con = sqlite3.connect("C:\MA_Pethke_3992454\MA_3992454_idea_1.db")
cur = con.cursor()

In [ ]:
## Tabellen Schema erstellen
#------------------- Tabelle Dokument --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Dokument(PMID INTEGER UNIQUE, titel TEXT, pubdate DATE)")

#------------------- Tabelle Zitationen --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Zitationen(PMID INTEGER, jahr YEAR, citecount INTEGER)")

#------------------- Tabelle Mesh --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Mesh(UI VARCHAR UNIQUE, name TEXT, datum DATE)")

#------------------- Tabelle Abfrage --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Abfrage(QID INTEGER PRIMARY KEY AUTOINCREMENT, query TEXT, PMID INTEGER, ranking INTEGER, mesh VARCHAR)")
con.commit()

# MeSH Data

In [39]:
mesh = "aspirin"
limit = 10000 # API limited to first 10000 documents; esearch is a way to get all data, but u have to work on an unix environment https://dataguide.nlm.nih.gov/edirect/edirect-vs-e-utilities.html
requ = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=("+urllib.parse.quote(mesh)+"[MeSH+Terms])&retmax="+str(limit)+"&sort=relevance&retmode=json&offset=")
requ_json = json.loads(requ.text)
doksToMeSH_all = requ_json['esearchresult']['idlist']
countResult = requ_json['esearchresult']['count']

In [40]:
doksToMeSH_all = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=("+urllib.parse.quote(mesh)+"[MeSH+Terms])&RetMax="+urllib.parse.quote(str(countResult))+"&sort=relevance&retmode=json")
doksToMeSH_all_json = json.loads(doksToMeSH_all.text)
print(doksToMeSH_all_json['esearchresult']['idlist'])

['17523700', '26739313', '30562044', '27729421', '32159800', '36099932', '14995999', '287960', '1899486', '845363', '28881293', '30547552', '34166171', '22257089', '26201059', '36690213', '29855050', '36500502', '19630812', '31957518', '2030640', '9282411', '7805939', '8205012', '15736495', '4947388', '17131625', '15079767', '9851731', '32439654', '36325602', '9353620', '31939218', '17081097', '2191940', '9456577', '67396', '20632895', '31382791', '16919334', '2687944', '4121596', '20144820', '31764014', '2237525', '25467192', '30753849', '18589844', '8369590', '34952668', '35767174', '88576', '5909852', '4002586', '12652150', '7763529', '1349086', '6563761', '23158031', '32669035', '31659060', '22494617', '11395667', '1485363', '7805078', '3292193', '35904749', '1479968', '36859712', '3142627', '2286215', '2064760', '22195683', '12007083', '36847696', '2653681', '1244713', '4189625', '4118703', '5450615', '7922246', '22297838', '11627349', '3181473', '3398469', '33967038', '11213864',